In [1]:
import _pickle as pickle
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import timeit

from tqdm.notebook import tqdm

In [2]:
stats_keys = [
    "accuracy",
    "p_iwl",
    "context contains query class",
    "ic_pred",
    "iw_pred",
    "num p_iwl >= 0.5",
    "p_iwl given context contains query class",
    "loss"
    # "similarity",
]

# Transformer
stats_keys = [
    "accuracy",
    "context contains query class",
    "loss"
]

In [3]:
repo_path = "/Users/chanb/research/ualberta/simple_icl"
results_dir = "/Users/chanb/research/ualberta/simple_icl/results/simple_icl/evaluation_results"

# repo_path = "/home/chanb/src/simple_icl"
# results_dir = "/home/chanb/scratch/simple_icl/results"

In [4]:
# variant_name = "simple_icl-fixed_g"
variant_name = "simple_icl-learned_g"
# variant_name = "simple_icl-transformer"
variant_name = "simple_icl-transformer-non_linear"

os.makedirs(
    os.path.join(repo_path, "cc_utils/plots/agg_stats"),
    exist_ok=True
)
results_dir = os.path.join(results_dir, variant_name)
results_dir

'/Users/chanb/research/ualberta/simple_icl/results/simple_icl/evaluation_results/simple_icl-transformer-non_linear'

In [7]:
stats_file = os.path.join(repo_path, "cc_utils/plots/agg_stats", "{}.feather".format(variant_name))
if os.path.isfile(stats_file):
    stats = pd.read_feather(stats_file)
else:
    stats = []

    if repo_path.startswith("/Users"):
        it = tqdm(os.listdir(results_dir))
    else:
        it = os.listdir(results_dir)

    for run_name in it:
        tic = timeit.default_timer()

        prefix = "-".join(run_name.split("-")[:-9])
        seed = run_name.split("-")[-9]

        data = pickle.load(open(os.path.join(results_dir, run_name), "rb"))
        for eval_name in data["stats"]:
            for stats_key in stats_keys:
                factor = 1
                if stats_key == "accuracy":
                    factor = 1 / 100

                stats.append(
                    dict(
                        variant=prefix,
                        seed=seed,
                        eval_name=eval_name,
                        stats_key=stats_key,
                        stats=[float(val) * factor for val in data["stats"][eval_name][stats_key][:400]],
                        **{
                            "_".join(key_val.split("_")[:-1]): key_val.split("_")[-1] for key_val in prefix.split("-")
                        },
                    )
                )

        toc = timeit.default_timer()
    stats = pd.DataFrame(stats)
    stats.to_feather(os.path.join(repo_path, "cc_utils/plots/agg_stats", "{}.feather".format(variant_name)))

  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
len(stats)

In [ ]:
stats.info()

In [ ]:
stats